# Scatterplot
Een scatterplot wordt gebruikt om het verband aan te tonen tussen bepaalde eigenschappen. In dit voorbeeld kijken we naar het verband tussen Wiskundescore en Leesscore vs Schrijfscore in de Students performance dataset

In [3]:
from pathlib import Path
from zipfile import ZipFile
import requests
URL = "https://www.kaggle.com/api/v1/datasets/download/spscientist/students-performance-in-exams"
STUDENTS_EXAM_ZIP = "students_performance_in_exams.zip"
students_exam_path = Path(STUDENTS_EXAM_ZIP)
if not students_exam_path.exists():
    data = requests.get(URL)
    with open(STUDENTS_EXAM_ZIP, "wb") as f:
        f.write(data.content)
    with open(STUDENTS_EXAM_ZIP, "rb") as f:
        ZipFile(f).extractall()

## Dataset inlezen en bekijken

In [ ]:
import pandas as pd
STUDENTS_CSV = "StudentsPerformance.csv"
df = pd.read_csv(STUDENTS_CSV, usecols=['math score', 'reading score', 'writing score', 'gender'], dtype={'gender':'category'})
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

## Wat weten we over een categorietype
Een categorietype zorgt ervoor dat elke categorie gekoppeld wordt aan een getal. Alleen dat getal wordt bijgehouden in de kolom. Aangezien geslachten niet echt een volgorde hebben, is 'ordered' False. Wanneer we een categorie met waarden 'slecht', 'middelmatig', 'goed' zouden hebben, heeft 'ordered' wel zin.


In [ ]:
print("df['gender'].cat.codes:",df['gender'].cat.codes, sep='\n')
print(f'{df['gender'].cat.categories=}')
print(f'{df['gender'].cat.ordered=}')


## Maak een scatterplot
Uit de scatterplot blijkt dat er een lineair verband is tussen wiskundescore en leesscore, zowel bij mannen als bij vrouwen.

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
plt.scatter(df.loc[df['gender']=='female','math score'], df.loc[df['gender']=='female','reading score'],
            c='pink', label='vrouwen', alpha=0.6)
plt.scatter(df.loc[df['gender']=='male','math score'], df.loc[df['gender']=='male','reading score'],
            c='blue', label='mannen', alpha=0.6)
plt.xlabel('wiskundescore', fontsize=10)
plt.ylabel('leesscore', fontsize=10)
plt.title('Relatie tussen wiskunde- en leesscore', fontsize=16)
plt.legend()
plt.show()

## Scattergrafiek maken met colormap
Wanneer we een derde dimensie willen toevoegen (schrijfscore), kunnen we kleuren gebruiken. Om de betekenis te tonen van de kleuren, gebruiken we een colormap. Om ervoor te zorgen dat de bolletjes volledig worden getoond, veranderen we de limieten.

We geven een alpha-waarde kleiner dan 1 mee om overlappende waarden te tonen. 

In [ ]:

plt.figure(figsize=(8, 6))
scatter = plt.scatter(df["math score"], df["reading score"], c=df["writing score"], cmap='RdYlGn', alpha=0.7, s=60, label="Studenten")
plt.xlabel("Wiskundescore", fontsize=12)
plt.ylabel("Leesscore", fontsize=12)
plt.title("Relatie tussen wiskunde- en leesscore\n(kleur=schrijfscores)", fontsize=14)
plt.xlim(-2, 102)
plt.ylim(-2, 102)
plt.legend()
cbar = plt.colorbar(scatter)
cbar.set_label('Schrijfscores', fontsize=12)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_position('zero')
plt.gca().spines['bottom'].set_position('zero')
plt.show()

## Colormap omkeren
Rood en groen zijn misschien niet de beste kleuren voor een kleurenmap. Maar het idee is dat scores boven 50 groen zijn, en scores beneden 50 rood. We kunnen een colormap omkeren door '_r' toe te voegen.

In [ ]:
plt.figure(figsize=(8, 6))
scatter = plt.scatter(df["math score"], df["reading score"], c=df["writing score"], cmap='RdYlGn_r', alpha=0.7, s=60, label="Studenten")
plt.xlabel("Wiskundescore", fontsize=12)
plt.ylabel("Leesscore", fontsize=12)
plt.title("Relatie tussen wiskunde- en leesscore\n(kleur=schrijfscores)", fontsize=14)
plt.xlim(-2, 102)
plt.ylim(-2, 102)
plt.legend()
cbar = plt.colorbar(scatter)
cbar.set_label('Schrijfscores', fontsize=12)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_position('zero')
plt.gca().spines['bottom'].set_position('zero')
plt.show()